In [14]:
import torch
import torch.nn as nn
from model import Net

In [9]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [11]:
input = torch.randn(1, 1, 32, 32) # B, C, H, W
print(input)
out = net(input)
print(out)

tensor([[[[-8.3821e-01, -8.1556e-01,  1.0019e-04,  ..., -5.2432e-01,
           -1.0921e+00,  1.0131e+00],
          [-1.2070e+00,  9.8689e-01, -4.8724e-01,  ..., -1.2867e+00,
            7.6329e-01, -1.4137e+00],
          [-6.3035e-01,  1.6757e+00,  1.3561e+00,  ..., -1.5670e+00,
            1.2329e+00,  4.6873e-01],
          ...,
          [ 1.7299e+00,  1.5903e-01,  7.1665e-01,  ..., -1.0144e+00,
           -1.7354e-01,  6.2239e-01],
          [ 1.3023e+00, -3.0101e-01,  5.7735e-02,  ...,  6.7889e-01,
           -3.9675e-01,  1.4835e+00],
          [-8.8434e-01,  9.4361e-01,  5.3870e-01,  ...,  6.3357e-01,
           -1.7556e+00, -1.8711e-01]]]])
tensor([[-0.0897, -0.1015,  0.0028,  0.0243, -0.0007,  0.0753,  0.0364, -0.0608,
          0.0200,  0.1342]], grad_fn=<AddmmBackward>)


In [12]:
# Zero the gradient buffers of all parameters and backprops with random gradients
net.zero_grad()
out.backward(torch.randn(1, 10))

In [15]:
input = torch.randn(1, 1, 32, 32) # B, C, H, W
print(input)
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output

criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor([[[[ 2.7878,  0.3462, -0.7569,  ...,  2.5178,  0.6197, -0.7230],
          [-0.3929, -0.0565,  2.1803,  ..., -0.1362,  1.4708,  0.5572],
          [ 0.0145,  1.3111, -0.6597,  ..., -0.2026, -0.0397,  0.5164],
          ...,
          [ 1.6938,  0.5123, -1.3229,  ...,  0.6201,  2.1661,  0.0901],
          [ 0.8073, -1.2730,  0.8813,  ...,  0.3938,  0.4871, -1.0443],
          [ 1.1810,  0.5785, -1.4296,  ..., -1.0398,  0.0143,  1.5348]]]])
tensor(1.1315, grad_fn=<MseLossBackward>)


In [16]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [17]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward() # 这样就能改变网络的权重

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0060, -0.0048,  0.0043,  0.0019, -0.0191, -0.0124])


PyTorch是通过optim模块更新权重（即学习）

In [26]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01) # 这里 optimizer 接受了网络 net对象 作为输入参数

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

print('conv1.bias.grad after optimization')
print(net.conv1.bias.grad)

conv1.bias.grad after optimization
tensor([-0.0295, -0.0017, -0.0052, -0.0169, -0.0210, -0.0188])
